In [1]:
import numpy as np
import cplex
import random

In [6]:
class Subproblem_LP():
    def __init__(self):
        self.n_dim = (n_stage - 1) * n_job * n_job
        self.n_constraints = 2 * (n_stage - 1) * n_job
        self.variable_names = ['x'+ str(i) for i in range(self.n_dim)]
        self.constraints_rows = self.set_constraints_byrows()
        self.obj = self.compute_obj()
        self.rhs = [1] * self.n_constraints
        self.senses = 'E'* self.n_constraints
        self.Subproblem_LP_TU = cplex.Cplex()
        self.opt_solution = []
        
    def set_job_to_position(self):
        job_to_position_onlyone = []
        for i in range((n_stage - 1) * n_job):
            job_to_position_onlyone.append([self.variable_names[i * n_job : (i + 1) * n_job], [1] * n_job])
        return job_to_position_onlyone
    
    def set_position_to_job(self):
        position_to_job_onlyone = []
        for j in range(n_stage - 1):
            for i in range(n_job):
                position_to_job_onlyone.append([self.variable_names[j * (n_job**2) + i: (j + 1) * (n_job**2): n_job], [1] * n_job])
        return position_to_job_onlyone
    
    def set_constraints_byrows(self):
        job_to_position_onlyone, position_to_job_onlyone = self.set_job_to_position(), self.set_position_to_job()
        return job_to_position_onlyone + position_to_job_onlyone
    
    def compute_obj(self, tval, lamd):
        return [random.random() for i in range(self.n_dim)]
    
    def compute_F6_x(self, tval, lamd):
        print(tval)
        print(self.opt_solution[0:6])
        print(self.opt_solution[6:12])
        print(self.opt_solution[12:18])
        print(self.opt_solution[18:24])
        print(self.opt_solution[24:30])
        print(self.opt_solution[30:36])
        print(self.opt_solution[36:42])
        print(self.opt_solution[42:48])
        print(ProcessTime)
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                                t5 = i1 + j * n_job
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t1] -  tval[t5]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) - 0.5 * (2 * ProcessTime[t5] + 1) * (self.opt_solution[t4] - self.opt_solution[t2]))           
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t5] -  tval[t1]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) + 0.5 * (2 * ProcessTime[t1] + 1) * (self.opt_solution[t2] - self.opt_solution[t4]))           
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t6 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                x_coeff[t2] += lamd[t3] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t4])**2 - self.opt_solution[t2] - self.opt_solution[t4]) + tval[t5] -  tval[t1]) * (self.opt_solution[t2] + self.opt_solution[t4] - 0.5) + 0.5 * (2 * ProcessTime[t1] + 1) * (self.opt_solution[t2] - self.opt_solution[t4]))
                                x_coeff[t2] += lamd[t6] * ((0.5 * ((self.opt_solution[t2] + self.opt_solution[t7])**2 - self.opt_solution[t2] - self.opt_solution[t7]) + tval[t1] -  tval[t5]) * (self.opt_solution[t2] + self.opt_solution[t7] - 0.5) - 0.5 * (2 * ProcessTime[t5] + 1) * (self.opt_solution[t7] - self.opt_solution[t2]))           
        return x_coeff

    def compute_f4_x(self, lamd):
        x_coeff = [0 for i in range((n_stage - 1) * n_job * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + n_batch * pcast - 1
                                x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * lamd[t3]
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast
                                x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * lamd[t3]
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                x_coeff[t2] += 0.25 * (2 * ProcessTime[t1] + 1) * (lamd[t3] + lamd[t4])
        return x_coeff

    def set_method(self, method):
        alg = self.Subproblem_LP_TU.parameters.lpmethod.values

        if method == "o":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.auto)
        elif method == "p":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.primal)
        elif method == "d":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.dual)
        elif method == "b":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.barrier)
            self.Subproblem_LP_TU.parameters.barrier.crossover.set(self.Subproblem_LP_TU.parameters.parameters.barrier.crossover.values.none)
        elif method == "h":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.barrier)
        elif method == "s":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.sifting)
        elif method == "c":
            self.Subproblem_LP_TU.parameters.lpmethod.set(alg.concurrent)
        else:
            raise ValueError('method must be one of "o", "p", "d", "b", "h", "s" or "c"')
        
    def solve(self, method = 'o'):
        self.Subproblem_LP_TU.objective.set_sense(self.Subproblem_LP_TU.objective.sense.maximize)
        ub = [1] * self.n_dim
    
        self.Subproblem_LP_TU.variables.add(obj = self.obj, ub = ub, names = self.variable_names)    
    
        assert(self.n_constraints == len(self.constraints_rows))
        row_names = ['c'+ str(i) for i in range(self.n_constraints)]
        print("the dimension of x:", self.n_dim, "the number of constraints:", self.n_constraints)
        self.Subproblem_LP_TU.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = row_names)
        self.set_method(method)
        self.Subproblem_LP_TU.solve()
        self.get_solvable_status()
        self.get_optvalues()
        
    def write_model(self):
        self.Subproblem_LP_TU.write('Subproblem_LP.lp')
        
    def get_optvalues(self):
        for i, x in enumerate(self.Subproblem_LP_TU.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.Subproblem_LP_TU.solution.get_status()
        if status == self.Subproblem_LP_TU.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.Subproblem_LP_TU.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.Subproblem_LP_TU.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def output_LP_result(self):
        with open('Subproblem_LP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
            f.write('\n')

In [ ]:
subproblem_lp = Subproblem_LP() 
job_to_position_onlyone = subproblem_lp.set_job_to_position()
print(len(job_to_position_onlyone))
print(job_to_position_onlyone)

In [ ]:
subproblem_lp = Subproblem_LP() 
position_to_job_onlyone = subproblem_lp.set_position_to_job()
print(len(position_to_job_onlyone))
print(position_to_job_onlyone)

In [7]:
n_stage, pcast, n_batch, n_machine_perstage = 3, 1, 3, 2
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']
assert(len(stage_name) == n_stage)

print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of job per cast:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)

n_position = dict(zip(stage_name[0:n_stage - 1], [n_job / n_machine['steelmaking'] ] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

subproblem_lp = Subproblem_LP() 

tval = [random.randint(0,1) for i in range((n_stage - 1) * n_job)]
lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
ProcessTime = [random.randint(0,1) for i in range((n_stage - 1) * n_job)]
subproblem_lp.opt_solution = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]

x_coeff = subproblem_lp.compute_F6_x(tval, lamd)
print(x_coeff)

lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
ProcessTime = [random.randint(30,36) for i in range((n_stage - 1) * n_job)]
print(subproblem_lp.compute_f4_x(lamd))

Number of stage: 3
Cast number of per machine: 1
Number of job per cast: 3
Number of machine: {'steelmaking': 2, 'refining': 2, 'casting': 2}
Number of job: 6
n_position =  {'steelmaking': 3.0, 'refining': 3.0}
[0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 1]
[1, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 1]
[1, 0, 0, 1, 0, 1]
[1, 1, 1, 1, 1, 1]
[0, 1, 0, 0, 1, 1]
[0, 1, 0, 0, 0, 1]
[0, 0, 1, 1, 0, 1]
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0]
[-4.0, -7.0, -5.0, 11.5, -12.5, 5.5, 6.5, 18.0, -5.0, 6.5, -13.5, -3.5, -1.0, -4.0, -5.0, -0.5, -8.5, 10.5, 12.5, -7.0, -5.0, 11.5, -12.5, 5.5, 12.5, 16.0, 4.5, 11.5, 18.0, 5.5, -4.0, 18.0, -5.0, -2.5, 17.0, 10.5, -4.5, 18.0, -3.5, -4.0, -10.0, 7.0, -4.5, -12.0, 12.0, 5.0, -12.0, 12.0, 11.0, -10.0, 6.0, 10.0, -10.0, 7.0, 11.0, 18.0, 6.0, -4.0, 18.0, -1.0, -1.5, -5.0, 6.0, 6.0, 17.0, 7.0, 2.0, 15.0, 12.0, -1.0, -7.0, 12.0]
[94.5, 189.0, 94.5, 94.5, 189.0, 94.5, 109.5, 219.0, 109.5, 109.5, 219.0, 109.5, 97.5, 195.0, 97.5, 97.5, 195.0, 97.5, 94.5, 189.0, 94.5,

In [68]:
class Subproblem_QP():
    def __init__(self):
        self.n_dim = n_job * n_stage + 2 * pcast * n_machine['steelmaking']
        self.variable_names = ['t' + str(i)  for i in range(self.n_dim)]
        self.Subproblem_QP = cplex.Cplex()
        self.duedate = self.compute_duedate()
        self.constraints_rows, self.rhs, self.n_per_constraints = self.set_constraints_byrows()
        self.senses = self.set_senses()
        self.obj_linear, self.qmat = self.compute_obj_linear(), self.compute_obj_qmat()
        self.opt_solution = []
        self.n_constraints = len(self.constraints_rows)
        
    def set_preceding_transfer_constraints(self):
        preceding_transfer_constraints = []
        for i in range(n_job * (n_stage - 1)):
            preceding_transfer_constraints.append([[self.variable_names[i], self.variable_names[i + n_job]], [1, -1]])

        rhs_transittime, rhs_processtime = [], processtime[0: n_job * (n_stage - 1)]
        for key in transittime:
            rhs_transittime += [transittime[key]] * n_job
        rhs = [ - processtime[i] - rhs_transittime[i] for i in range(n_job * (n_stage - 1))]
        return preceding_transfer_constraints, rhs, rhs_processtime, rhs_transittime

    def set_process_continuous_constraints(self):
        process_continuous, adjacent_casts = [], []
        rhs_continuous_processtime, rhs_adjacent_casts = [], []
        assert(n_job * (n_stage - 1) % n_batch == 0)
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if (i + 1) % n_batch != 0:
                process_continuous.append([[self.variable_names[i], self.variable_names[i + 1]], [-1, 1]])
                rhs_continuous_processtime.append(processtime[i])
            else:
                if (i + 1) % (n_batch * pcast) != 0:
                    adjacent_casts.append([[self.variable_names[i], self.variable_names[i + 1]], [-1, 1]])
                    rhs_adjacent_casts.append(- processtime[i] - su)
        return process_continuous, adjacent_casts, rhs_continuous_processtime, rhs_adjacent_casts 

    def set_additional_constraints(self):
        additional_constraints = []
        index = n_job * n_stage
        index_start = pcast * n_machine['casting']
        assert(n_job * (n_stage - 1) % n_batch == 0 and (index % n_batch == 0))
        batch = 0
        for i in range(n_job * (n_stage - 1), n_job * n_stage - 1):
            if i % n_batch == 0:
                additional_constraints.append([[self.variable_names[i], self.variable_names[index + batch], self.variable_names[index + batch + index_start]], [1, -1, 1]])
                batch += 1
        return additional_constraints
    
    def set_constraints_byrows(self):
        preceding_transfer_constraints, rhs_preceding_transfer, _, _ = self.set_preceding_transfer_constraints()
        process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts = self.set_process_continuous_constraints()
        additional_constraints = self.set_additional_constraints()
    
        rows = preceding_transfer_constraints + process_continuous_constraints + adjacent_casts_constraints + additional_constraints
        
        n_per_constraints = [len(preceding_transfer_constraints), len(process_continuous_constraints), len(adjacent_casts_constraints), len(additional_constraints)]
        
        rhs = rhs_preceding_transfer + rhs_continuous_processtime + rhs_adjacent_casts + self.duedate
        return rows, rhs, n_per_constraints
    
    def set_senses(self):
        s, senses = 'LELE', ''
        for i in range(len(self.n_per_constraints)):
            senses += s[i] * self.n_per_constraints[i]
        return senses
    
    def compute_duedate(self):
        duedate = [0 for i in range(pcast * n_machine['casting'])]
        duedate_start = [0 for i in range(n_machine['casting'])]
        n_cast = pcast * n_machine['casting']
        for i in range(n_machine['casting']):
            index = 0
            for key in transittime:
                duedate_start[i] += processtime[i * n_cast + index * n_job] + transittime[key]
                index += 1
        print(duedate_start)
    
        for i in range(n_machine['casting']):
            duedate[i * pcast] = duedate_start[i]
            index_start = n_job * (n_stage - 1) + i * n_cast
            for j in range(1, pcast):
                duedate[i * pcast + j] = duedate[i * pcast + j - 1] + sum(processtime[index_start: index_start + n_batch]) + su
                index_start = index_start + n_batch
        return duedate
    
    def compute_obj_wait_time(self):
        obj_wait_time = np.zeros((n_stage, n_job))
        for i in range(n_stage - 1):
            for j in range(n_job):
                obj_wait_time[i + 1][j], obj_wait_time[i][j] = obj_wait_time[i + 1][j] + coeff[i], obj_wait_time[i][j] - coeff[i]
        obj_wait_time = obj_wait_time.reshape(1, n_stage * n_job)
        return np.hstack((obj_wait_time, np.zeros((1, self.n_dim - n_stage * n_job))))

    def compute_obj_casting(self):
        obj_casting = np.hstack((d_coeff['d1'] *  np.ones((1, pcast * n_machine['casting'])), d_coeff['d2'] * np.ones((1, pcast * n_machine['casting']))))
        obj_casting = np.hstack((np.zeros((1, n_job * n_stage)), obj_casting))
        return obj_casting
    
    def compute_obj_linear(self):
        return []
    
    def compute_obj_qmat(self):
        qmat = []
        for i in range(self.n_dim):
            qmat.append([self.variable_names, [- 1 for i in range(self.n_dim)]])
        return qmat
    
    def compute_F6_t(self, xval, lamd):
        print(xval[0:6])
        print(xval[6:12])
        print(xval[12:18])
        print(xval[18:24])
        print(xval[24:30])
        print(xval[30:36])
        print(xval[36:42])
        print(xval[42:48])
        print(xval[48:54])
        print(xval[54:60])
        print(xval[60:66])
        print(xval[66:72])
        t_coeff = [0 for i in range((n_stage - 1) * n_job)]
        for i in range(n_job):
            for j in range(n_stage - 1):
                t1 = i + j * n_job
                for r in range(n_batch * pcast):
                    for k in range(n_machine_perstage):
                        t2 = i * n_job + j * n_job * n_job + k * n_batch * pcast + r
                        for i1 in range(n_job):
                            if r == n_batch * pcast - 1:
                                t3 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1 
                                t5 = i1 + j * n_job
                                t_coeff[t1] += lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t1] -  self.opt_solution[t5])          
                            elif r == 0:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t_coeff[t1] -= lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t5] -  self.opt_solution[t1])
                            else:
                                t3 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                                t4 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r + 1
                                t5 = i1 + j * n_job
                                t6 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t7 = i1 * n_job + j * n_job * n_job + k * n_batch * pcast + r - 1
                                t_coeff[t1] -= lamd[t3] * (0.5 * ((xval[t2] + xval[t4])**2 - xval[t2] - xval[t4]) + self.opt_solution[t5] -  self.opt_solution[t1]) 
                                t_coeff[t1] += lamd[t6] * (0.5 * ((xval[t2] + xval[t7])**2 - xval[t2] - xval[t7]) + self.opt_solution[t1] -  self.opt_solution[t5])            
        return t_coeff + [0] * (self.n_dim - len(t_coeff))
    
    def compute_qmat(self, lamd):
        quad_t_coeff = np.zeros((self.n_dim, self.n_dim))
        for i in range(n_job):
            for j in range(n_stage - 1):
                t3 = i + j * n_job
                for i1 in range(n_job):
                    t4 = i1 + j * n_job 
                    for r in range(n_batch * pcast):
                        for k in range(n_machine_perstage):
                            t1 = i * n_job + i1 * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            t2 = i1 * n_job + i * (n_stage - 1) * n_job * n_job + j * n_job * n_job + k * n_batch * pcast + r
                            if t3 == t4:
                                 quad_t_coeff[t3,t4] += 0.5 * (lamd[t1] + lamd[t2])
                            else:
                                 quad_t_coeff[t3,t4] -= 0.5 * (lamd[t1] + lamd[t2])
                                    
        quad_t_coeff = quad_t_coeff.tolist()
        return [[self.variable_names, quad_t_coeff[i]] for i in range(len(quad_t_coeff))]
    
    def solve(self):
        self.Subproblem_QP.objective.set_sense(self.Subproblem_QP.objective.sense.maximize)
        
        ub   = [cplex.infinity] * self.n_dim
        self.Subproblem_QP.variables.add(obj = self.obj_linear, ub = ub, names = self.variable_names)
        self.Subproblem_QP.objective.set_quadratic(self.qmat)
    
        rownames = ['c' + str(i)  for i in range(self.n_constraints)]

        self.Subproblem_QP.linear_constraints.add(lin_expr = self.constraints_rows, senses = self.senses, rhs = self.rhs, names = rownames)
        self.Subproblem_QP.solve()
        self.get_solvable_status()
        self.get_optvalues()

    def write_model(self):
        self.Subproblem_QP.write('Subproblem_QP.lp')
        
    def get_optvalues(self):
        for i, x in enumerate(self.Subproblem_QP.solution.get_values()):
            if i < self.n_dim:
                self.opt_solution.append(x)
                
    def get_solvable_status(self):
        status = self.Subproblem_QP.solution.get_status()
        if status == self.Subproblem_QP.solution.status.unbounded:
            print("Model is unbounded")
        if status == self.Subproblem_QP.solution.status.infeasible:
            print("Model is infeasible")
        if status == self.Subproblem_QP.solution.status.infeasible_or_unbounded:
            print("Model is infeasible or unbounded")
            
    def output_QP_result(self):
        with open('Subproblem_QP_result.txt', 'w') as f:
            for i in range(n_job * n_stage):
                f.write(str(self.opt_solution[i * n_job:(i + 1) * n_job]))
            f.write('\n')

In [ ]:
subproblem_qp= Subproblem_QP()
preceding_transfer_constraints, rhs_preceding_transfer, rhs_processtime, rhs_transittime = subproblem_qp.set_preceding_transfer_constraints()
print(preceding_transfer_constraints)
print(rhs_processtime)
print(len(rhs_processtime))
print(rhs_transittime)
print(len(rhs_transittime))
print(rhs_preceding_transfer)
print(len(rhs_preceding_transfer))

In [ ]:
subproblem_qp = Subproblem_QP()
process_continuous_constraints, adjacent_casts_constraints, rhs_continuous_processtime, rhs_adjacent_casts =  subproblem_qp.set_process_continuous_constraints()
print(process_continuous_constraints)
print(rhs_continuous_processtime)
print(adjacent_casts_constraints)
print(rhs_adjacent_casts)

In [ ]:
subproblem_qp = Subproblem_QP()
additional_constraints = subproblem_qp.set_additional_constraints()
print(additional_constraints)

In [ ]:
subproblem_qp = Subproblem_QP()
duedate = subproblem_qp.compute_duedate()
print(duedate)
print(len(duedate))
print(processtime)

In [ ]:
subproblem_qp = Subproblem_QP()
obj_wait_time = subproblem_qp.compute_obj_wait_time()
print(obj_wait_time)
print(obj_wait_time.shape)

In [ ]:
subproblem_qp = Subproblem_QP()
obj_casting = subproblem_qp.compute_obj_casting()
print(obj_casting)
print(obj_casting.shape)

In [70]:
n_stage, pcast, n_batch, n_machine_perstage = 3, 1, 3, 2
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']
assert(len(stage_name) == n_stage)

print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of job per cast:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)

n_position = dict(zip(stage_name[0:n_stage - 1], [n_job / n_machine['steelmaking'] ] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

Number of stage: 3
Cast number of per machine: 1
Number of job per cast: 3
Number of machine: {'steelmaking': 2, 'refining': 2, 'casting': 2}
Number of job: 6
n_position =  {'steelmaking': 3.0, 'refining': 3.0}


In [73]:
subproblem_qp = Subproblem_QP()
subproblem_qp.opt_solution = [random.randint(0,1) for i in range((n_stage - 1) * n_job)]
print(subproblem_qp.opt_solution)
lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
xval = [random.randint(0,1) for i in range((n_stage - 1) * n_job * n_job)]
t_coeff = subproblem_qp.compute_F6_t()
print(len(t_coeff))

[88.55318232974672, 96.7463255217378]
[1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1]
[1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1]
[0, 1, 0, 1, 0, 1]
[1, 1, 1, 1, 1, 1]
[1, 0, 0, 0, 1, 1]
[0, 1, 1, 1, 0, 0]
[0, 1, 0, 1, 0, 0]
[1, 0, 1, 0, 1, 0]
[0, 1, 1, 1, 0, 1]
[0, 0, 1, 0, 1, 0]
[1, 1, 1, 1, 1, 0]
[0, 1, 0, 1, 1, 0]
[0, 0, 1, 1, 1, 1]
[0, 0, 1, 1, 0, 1]
22


In [74]:
lamd = [1 for i in range((n_stage - 1) * n_job * n_job * n_job)]
qmat = subproblem_qp.compute_qmat()
print(qmat)
print(len(qmat))

[[['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21'], [6.0, -6.0, -6.0, -6.0, -6.0, -6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21'], [-6.0, 6.0, -6.0, -6.0, -6.0, -6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21'], [-6.0, -6.0, 6.0, -6.0, -6.0, -6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], [['t0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21'], [-6.0, -6.0, -6.0, 6.0, -6.0, -6.0, 0.0, 0.0, 0.0, 0.0, 0

In [11]:
n_stage, pcast, n_batch, n_machine_perstage = 3, 5, 3, 3
stage_name = ['steelmaking', 'refining', 'casting']
n_machine = dict(zip(stage_name, [n_machine_perstage for i in range(n_stage)]))
n_job = pcast * n_batch * n_machine['steelmaking']

processtime = [14 * random.random() + 36 for i in range(n_job * n_stage)]
transittime = dict(zip([stage_name[i] + 'to' + stage_name[i+1] for i in range(n_stage - 1)], [3 * random.random() + 3 for i in range(n_stage - 1)]))
su = 80
duedate = [0 for i in range(pcast * n_machine['casting'])]

penalty_coeff = 10
d_coeff = {'d1': 10, 'd2': 110}
coeff = [10 + 20 * i for i in range(n_stage - 1)]

assert(len(stage_name) == n_stage)
print("Number of stage:", n_stage)
print("Cast number of per machine:", pcast)
print("Number of machine per stage:", n_batch)
print('Number of machine:', n_machine)
print('Number of job:', n_job)
print('transfer time:', transittime)

n_position = dict(zip(stage_name[0:n_stage - 1], [n_job / n_machine['steelmaking'] ] * (n_stage - 1)))
print('n_position = ', n_position)
assert(len(n_machine) == n_stage)
assert(len(n_position) == n_stage - 1)

Number of stage: 3
Cast number of per machine: 5
Number of machine per stage: 3
Number of machine: {'steelmaking': 3, 'refining': 3, 'casting': 3}
Number of job: 45
transfer time: {'steelmakingtorefining': 3.4797461106018197, 'refiningtocasting': 4.827185726511883}
n_position =  {'steelmaking': 15.0, 'refining': 15.0}


In [ ]:
m, r = 2, 0.01
lambd = np.zeros((n_job, n_job, n_job, n_stage))
maxiteration_times = 10
c0 = 1
g0 = np.ones((n_job, n_job, n_job, n_stage))
gm = compute_subgradients()
for iteration_times in range(1, maxiteration_times):
    stepsize = compute_stepsize(iteration_times, gm)
    print(stepsize)

In [ ]:
def compute_subgradients():
    return np.ones((lambd.shape))

def compute_stepsize(i, gm):
    p = 1 - 1/(i ** r)
    alpha = 1
    for j in range(1, i + 1):
        alpha = alpha * (1 - 1/(m * j ** p))
    if np.linalg.norm(gm) != 0:
        c = alpha * c0 * np.linalg.norm(g0) / np.linalg.norm(gm)
    return c